# Create Solr Core

In [198]:
# Delete any existing core
!curl -X GET "http://localhost:8983/solr/admin/cores?action=UNLOAD&core=search_reddit&deleteInstanceDir=true&deleteDataDir=true"

{
  "responseHeader":{
    "status":0,
    "QTime":62
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    59  100    59    0     0    215      0 --:--:-- --:--:-- --:--:--   215


In [199]:
import shutil

# Create directory
shutil.copytree("C:\\solr-9.5.0-slim\\server\\solr\\configsets\\_default", "C:\\solr-9.5.0-slim\\server\\solr\\search_reddit")

'C:\\solr-9.5.0-slim\\server\\solr\\search_reddit'

In [200]:
# Create solr core with default settings
!curl -X GET "http://localhost:8983/solr/admin/cores?action=CREATE&name=search_reddit&instanceDir=search_reddit"

{
  "responseHeader":{
    "status":0,
    "QTime":151
  },
  "core":"search_reddit"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    86  100    86    0     0    240      0 --:--:-- --:--:-- --:--:--   240


# Solr recommends we use HTTP requests to edit schema. Because of data integrity issues

In [201]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{
  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  }]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   574  100   574    0     0   2753      0 --:--:-- --:--:-- --:--:--  2746


# Add custom schema & index

https://solr.apache.org/guide/6_6/field-properties-by-use-case.html#field-properties-by-use-case
https://solr.apache.org/guide/6_6/field-type-definitions-and-properties.html#field-type-definitions-and-properties

## With only basic cleaning done beforehand, the rest have to be done in Solr. Therefore we define a custom field type for text

- https://solr.apache.org/guide/6_6/filter-descriptions.html

text_general for reference
```
</fieldType>
  <fieldType name="text_general" class="solr.TextField" positionIncrementGap="100" multiValued="true">
    <analyzer type="index">
      <tokenizer name="standard"/>
      <filter words="stopwords.txt" name="stop" ignoreCase="true"/>
      <filter name="lowercase"/>
    </analyzer>
    <analyzer type="query">
      <tokenizer name="standard"/>
      <filter words="stopwords.txt" name="stop" ignoreCase="true"/>
      <filter ignoreCase="true" synonyms="synonyms.txt" name="synonymGraph" expand="true"/>
      <filter name="lowercase"/>
    </analyzer>
</fieldType>
```

text_reddit custom field type
- Query and index filters do not need to be the same
- Synonyms are used to expand search results for query terms only
- NGram filter is used to create n-grams for index terms only
- The rest can be the same as text_general
```
  <fieldType name="text_reddit" class="solr.TextField" positionIncrementGap="100" multiValued="true">
    <analyzer type="index">
      <tokenizer class="solr.WhitespaceTokenizerFactory"/>
      <filter class="solr.StopFilterFactory" words="stopwords.txt" ignoreCase="true"/>
      <filter class="solr.LowerCaseFilterFactory"/>
      <filter class="solr.SnowballPorterFilterFactory" language="English"/>
      <filter class="solr.NGramFilterFactory" maxGramSize="20" minGramSize="2"/>
    </analyzer>
    <analyzer type="query">
      <tokenizer class="solr.WhitespaceTokenizerFactory"/>
      <filter class="solr.StopFilterFactory" words="stopwords.txt" ignoreCase="true"/>
      <filter class="solr.SynonymGraphFilterFactory" ignoreCase="true" synonyms="synonyms.txt" expand="true"/>
      <filter class="solr.LowerCaseFilterFactory"/>
      <filter class="solr.SnowballPorterFilterFactory" language="English"/>
    </analyzer>
```

In [202]:
# Tried with General tokenizer but didnt work
# !curl -X POST -H "Content-type:application/json" --data-binary "{"add-field-type":{"name":"text_reddit","class":"solr.TextField","positionIncrementGap":"100","multiValued":true,"indexAnalyzer":{"tokenizer":{"class":"solr.StandardTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"},{"class":"solr.NGramFilterFactory","minGramSize":2,"maxGramSize":20}]},"queryAnalyzer":{"tokenizer":{"class":"solr.StandardTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.SynonymGraphFilterFactory","ignoreCase":true,"synonyms":"synonyms.txt","expand":true},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"}]}}}" http://localhost:8983/api/cores/search_reddit/schema

# Try with Whitespace tokenizer
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field-type":{"name":"text_reddit","class":"solr.TextField","positionIncrementGap":"100","multiValued":true,"indexAnalyzer":{"tokenizer":{"class":"solr.WhitespaceTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"},{"class":"solr.NGramFilterFactory","minGramSize":2,"maxGramSize":20}]},"queryAnalyzer":{"tokenizer":{"class":"solr.WhitespaceTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.SynonymGraphFilterFactory","ignoreCase":true,"synonyms":"synonyms.txt","expand":true},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"}]}}}" http://localhost:8983/api/cores/search_reddit/schema

{
  "responseHeader":{
    "status":0,
    "QTime":97
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   781  100    59  100   722    190   2332 --:--:-- --:--:-- --:--:--  2527


## Field types explained
- indexed -> If true, the value of the field can be used in queries to retrieve matching documents.
- stored -> If true, the actual value of the field can be retrieved by queries.

Our search engine primarily searches by comment & post texts

```
<field name="_nest_path_" type="_nest_path_"/>
<field name="_root_" type="string" indexed="true" stored="false" docValues="false"/>
<field name="_text_" type="text_general" multiValued="true" indexed="true" stored="false"/>
<field name="_version_" type="plong" indexed="false" stored="false"/>
<field name="author" type="text_general"/>
<field name="created_utc" type="pdates"/>
<field name="edited" type="pdates"/>
<field name="id" type="string" multiValued="false" indexed="true" required="true" stored="true"/>
<field name="num_comments" type="pdoubles"/>
<field name="permalink" type="text_general"/>
<field name="post_id" type="text_general"/>
<field name="score" type="plongs"/>
<field name="subreddit_name" type="text_general"/>
<field name="text" type="text_general"/>
<field name="type" type="text_general"/>
<field name="upvote_ratio" type="pdoubles"/>
<field name="url" type="text_general"/>
```

In [203]:
# Author, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"author","type":"string","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# Text, searched and retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"text","type":"text_reddit","stored":true,"indexed":true,"required":true}}" http://localhost:8983/api/cores/search_reddit/schema

# created_utc, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"created_utc","type":"pdates","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# edited, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"edited","type":"pdates","stored":true,"indexed":true}}" http://localhost:8983/api/cores/search_reddit/schema

# id, retrieved, unique key
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"id","type":"string","stored":true,"indexed":true,"multiValued":false,"required":true}}" http://localhost:8983/api/cores/search_reddit/schema

# num_comments, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"num_comments","type":"pdoubles","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# permalink, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"permalink","type":"string","stored":true,"multiValued":false}}" http://localhost:8983/api/cores/search_reddit/schema

# score, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"score","type":"plong","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# subreddit_name, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"subreddit_name","type":"string","stored":true,"multiValued":true}}" http://localhost:8983/api/cores/search_reddit/schema

# upvote_ratio, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"upvote_ratio","type":"pdoubles","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# url, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"url","type":"string","stored":true,"multiValued":false}}" http://localhost:8983/api/cores/search_reddit/schema

# type, retrieved, have duplicates
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"type","type":"string","stored":true,"multiValued":true}}" http://localhost:8983/api/cores/search_reddit/schema

# post_id, retrieved, have duplicates
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"post_id","type":"string","stored":true,"multiValued":true}}" http://localhost:8983/api/cores/search_reddit/schema

{
  "responseHeader":{
    "status":0,
    "QTime":88
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   171  100    59  100   112    190    360 --:--:-- --:--:-- --:--:--   551


{
  "responseHeader":{
    "status":0,
    "QTime":84
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   138  100    59  100    79    193    259 --:--:-- --:--:-- --:--:--   452
100   138  100    59  100    79    193    259 --:--:-- --:--:-- --:--:--   452


{
  "responseHeader":{
    "status":0,
    "QTime":89
  }
}


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   123  100    59  100    64    201    218 --:--:-- --:--:-- --:--:--   421


{
  "responseHeader":{
    "status":0,
    "QTime":95
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   123  100    59  100    64    187    203 --:--:-- --:--:-- --:--:--   390


{
  "responseHeader":{
    "status":0,
    "QTime":84
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   126  100    59  100    67    200    227 --:--:-- --:--:-- --:--:--   428


In [204]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{
  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"author",
    "type":"string",
    "omitNorms":true,
    "docValues":true,
    "multiValued":false,
    "indexed":true,
    "stored":true
  },{
    "name":"created_utc",
    "type":"pdates",
    "omitNorms":true,
    "docValues":true,
    "multiValued":false,
    "indexed":true,
    "stored":true
  },{
    "name":"edited",
    "type":"pdates",
    "indexed":true,
    "stored":true
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  },{
    "name":"num_comments"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2023  100  2023    0     0   9515      0 --:--:-- --:--:-- --:--:--  9542


# Ingest data

In [ ]:
# add document to solr
!curl "http://localhost:8983/solr/search_reddit/update?commit=true" --data-binary @data/cleaned_combined_data.csv -H "Content-type:application/csv"

# Test query data

- https://solr.apache.org/guide/6_6/the-standard-query-parser.html

## Select with keyword search

In [ ]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=text:congrats*&rows=10"

## Search with timeframe range filter

In [ ]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]&rows=10"

## Combination of conditions

In [ ]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=text:congrats* AND created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]&rows=10"